<a href="https://colab.research.google.com/github/NilakshanRaveendran/bilingual-voice-translation-research/blob/main/notebooks/English_Dataset_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

BASE_DIR = "/content/drive/MyDrive/bilingual_voice_translation"
RAW_DIR = f"{BASE_DIR}/data/raw"
PROC_DIR = f"{BASE_DIR}/data/processed"
SEG_DIR = f"{BASE_DIR}/data/segments"

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)
os.makedirs(SEG_DIR, exist_ok=True)

print("Folders created successfully")

Folders created successfully


In [3]:
!pip install librosa soundfile pydub noisereduce torchaudio wget datasets

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=9cd12f006d14f9ca0c2ea5e06a67425ca91f566c3e758ebbb04609e3d2452fa1
  Stored in directory: /root/.cache/pip/wheels/01/46/3b/e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget


In [4]:
import os

RAW_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/raw"
os.makedirs(RAW_DIR, exist_ok=True)

print("Raw data directory ready")

Raw data directory ready


In [5]:
!wget https://storage.googleapis.com/cvss/cvss_c_v1.0/cvss_c_es_en_v1.0.tar.gz

--2026-01-21 09:12:16--  https://storage.googleapis.com/cvss/cvss_c_v1.0/cvss_c_es_en_v1.0.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.118.207, 74.125.24.207, 172.217.194.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.118.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13592155358 (13G) [application/octet-stream]
Saving to: ‘cvss_c_es_en_v1.0.tar.gz’

cvss_c_es_en_v1.0.t 100%[===================>]  12.66G  21.9MB/s    in 9m 48s  

2026-01-21 09:22:05 (22.0 MB/s) - ‘cvss_c_es_en_v1.0.tar.gz’ saved [13592155358/13592155358]



In [6]:
!ls -lh cvss_c_es_en_v1.0.tar.gz

-rw-r--r-- 1 root root 13G Jan 11  2022 cvss_c_es_en_v1.0.tar.gz


In [7]:
!tar -xvzf cvss_c_es_en_v1.0.tar.gz

Streaming output truncated to the last 5000 lines.
test/common_voice_es_19712451.mp3.wav
test/common_voice_es_19712453.mp3.wav
test/common_voice_es_19712593.mp3.wav
test/common_voice_es_19712596.mp3.wav
test/common_voice_es_19712598.mp3.wav
test/common_voice_es_19712601.mp3.wav
test/common_voice_es_19712652.mp3.wav
test/common_voice_es_19712653.mp3.wav
test/common_voice_es_19712720.mp3.wav
test/common_voice_es_19712724.mp3.wav
test/common_voice_es_19712728.mp3.wav
test/common_voice_es_19712731.mp3.wav
test/common_voice_es_19712901.mp3.wav
test/common_voice_es_19712907.mp3.wav
test/common_voice_es_19712967.mp3.wav
test/common_voice_es_19712970.mp3.wav
test/common_voice_es_19712972.mp3.wav
test/common_voice_es_19713060.mp3.wav
test/common_voice_es_19713324.mp3.wav
test/common_voice_es_19713326.mp3.wav
test/common_voice_es_19713327.mp3.wav
test/common_voice_es_19713330.mp3.wav
test/common_voice_es_19713384.mp3.wav
test/common_voice_es_19713385.mp3.wav
test/common_voice_es_19713387.mp3.wav

In [8]:
ls /content/train | head -n 100 | xargs -I {} mv /content/train/{} /content/drive/MyDrive/bilingual_voice_translation/data/raw/

In [9]:
import os, glob, shutil

META_DIR = "/content/drive/MyDrive/bilingual_voice_translation/metadata"
os.makedirs(META_DIR, exist_ok=True)
print("Metadata folder:", META_DIR)

Metadata folder: /content/drive/MyDrive/bilingual_voice_translation/metadata


In [10]:
# Search for train/dev/test tsv anywhere under /content
tsv_candidates = glob.glob("/content/**/train.tsv", recursive=True) + \
                 glob.glob("/content/**/dev.tsv", recursive=True) + \
                 glob.glob("/content/**/test.tsv", recursive=True)

print("Found TSVs:")
for p in tsv_candidates:
    print(" -", p)

# Copy them to Drive metadata folder
for p in tsv_candidates:
    shutil.copy(p, os.path.join(META_DIR, os.path.basename(p)))

print("✅ Copied TSV files to:", META_DIR)
print("Now in Drive:", os.listdir(META_DIR))

Found TSVs:
 - /content/train.tsv
 - /content/dev.tsv
 - /content/test.tsv
✅ Copied TSV files to: /content/drive/MyDrive/bilingual_voice_translation/metadata
Now in Drive: ['train.tsv', 'dev.tsv', 'test.tsv']


In [11]:
import os

RAW_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/raw"
files = os.listdir(RAW_DIR)

print("Number of files in raw folder:", len(files))
print(files[:10])

Number of files in raw folder: 100
['common_voice_es_18306544.mp3.wav', 'common_voice_es_18306545.mp3.wav', 'common_voice_es_18306546.mp3.wav', 'common_voice_es_18306547.mp3.wav', 'common_voice_es_18306548.mp3.wav', 'common_voice_es_18306565.mp3.wav', 'common_voice_es_18306566.mp3.wav', 'common_voice_es_18306567.mp3.wav', 'common_voice_es_18306568.mp3.wav', 'common_voice_es_18306579.mp3.wav']


In [13]:
raw_keys = set()
for f in files:
    if f.endswith(".mp3.wav"):
        raw_keys.add(f.replace(".wav", ""))  # becomes *.mp3
    else:
        raw_keys.add(os.path.splitext(f)[0]) # fallback

print("Example keys:", list(raw_keys)[:5])

Example keys: ['common_voice_es_18307256.mp3', 'common_voice_es_18307258.mp3', 'common_voice_es_18306579.mp3', 'common_voice_es_18307307.mp3', 'common_voice_es_18307341.mp3']


In [14]:
import pandas as pd

train_path = os.path.join(META_DIR, "train.tsv")
df_train = pd.read_csv(train_path, sep="\t", header=None, names=["file", "text"])

# keep only rows whose file is in our raw_keys set
df_train_100 = df_train[df_train["file"].isin(raw_keys)].copy()

out_path = os.path.join(META_DIR, "train_filtered_100.tsv")
df_train_100.to_csv(out_path, sep="\t", index=False, header=False)

print("✅ Filtered TSV saved:", out_path)
print("Rows in filtered TSV:", len(df_train_100))
df_train_100.head()

✅ Filtered TSV saved: /content/drive/MyDrive/bilingual_voice_translation/metadata/train_filtered_100.tsv
Rows in filtered TSV: 100


,file,text
0,common_voice_es_18306544.mp3,but are their people available
1,common_voice_es_18306545.mp3,the priority is to ensure the safety of the ci...
2,common_voice_es_18306546.mp3,hey brunette
3,common_voice_es_18306547.mp3,it's obvious that you don't care anymore but
4,common_voice_es_18306548.mp3,there juvenile intruders i repeat myself recov...


In [15]:
import os

RAW_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/raw"
STD_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/standardized"
CLEAN_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/cleaned"
SEG_DIR = "/content/drive/MyDrive/bilingual_voice_translation/data/segments"

os.makedirs(STD_DIR, exist_ok=True)
os.makedirs(CLEAN_DIR, exist_ok=True)
os.makedirs(SEG_DIR, exist_ok=True)

print("Folders ready")

Folders ready


In [16]:
import librosa
import soundfile as sf

TARGET_SR = 16000

for file in os.listdir(RAW_DIR):
    if file.endswith(".wav"):
        in_path = os.path.join(RAW_DIR, file)
        out_path = os.path.join(STD_DIR, file)

        audio, sr = librosa.load(in_path, sr=None, mono=True)

        if sr != TARGET_SR:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)

        sf.write(out_path, audio, TARGET_SR)

print("✅ Audio standardization completed (16 kHz, mono)")

✅ Audio standardization completed (16 kHz, mono)


In [17]:
import noisereduce as nr
import numpy as np

for file in os.listdir(STD_DIR):
    in_path = os.path.join(STD_DIR, file)
    out_path = os.path.join(CLEAN_DIR, file)

    audio, sr = librosa.load(in_path, sr=16000)

    reduced = nr.reduce_noise(y=audio, sr=sr)

    # normalize
    if np.max(np.abs(reduced)) > 0:
        reduced = reduced / np.max(np.abs(reduced))

    sf.write(out_path, reduced, sr)

print("✅ Noise reduction & normalization completed")

✅ Noise reduction & normalization completed


In [18]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

def segment_audio(file_path, out_dir):
    audio = AudioSegment.from_wav(file_path)

    chunks = split_on_silence(
        audio,
        min_silence_len=500,     # 0.5 sec silence
        silence_thresh=-40,      # silence threshold (dB)
        keep_silence=200         # keep short pause
    )

    base = os.path.splitext(os.path.basename(file_path))[0]

    for i, chunk in enumerate(chunks):
        chunk.export(
            os.path.join(out_dir, f"{base}_seg_{i}.wav"),
            format="wav"
        )

for file in os.listdir(CLEAN_DIR):
    segment_audio(os.path.join(CLEAN_DIR, file), SEG_DIR)

print("✅ Silence-based segmentation completed")

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


✅ Silence-based segmentation completed


In [19]:
segments = os.listdir(SEG_DIR)

print("Total segments created:", len(segments))
print("Sample segment files:", segments[:10])

Total segments created: 103
Sample segment files: ['common_voice_es_18306544.mp3_seg_0.wav', 'common_voice_es_18306545.mp3_seg_0.wav', 'common_voice_es_18306546.mp3_seg_0.wav', 'common_voice_es_18306547.mp3_seg_0.wav', 'common_voice_es_18306548.mp3_seg_0.wav', 'common_voice_es_18306548.mp3_seg_1.wav', 'common_voice_es_18306565.mp3_seg_0.wav', 'common_voice_es_18306566.mp3_seg_0.wav', 'common_voice_es_18306567.mp3_seg_0.wav', 'common_voice_es_18306568.mp3_seg_0.wav']


In [20]:
import pandas as pd

rows = []

for f in segments:
    path = os.path.join(SEG_DIR, f)
    audio, sr = librosa.load(path, sr=16000)
    duration = librosa.get_duration(y=audio, sr=sr)
    rows.append([f, duration])

df = pd.DataFrame(rows, columns=["Segment", "Duration (seconds)"])
df.head()

,Segment,Duration (seconds)
0,common_voice_es_18306544.mp3_seg_0.wav,1.438
1,common_voice_es_18306545.mp3_seg_0.wav,3.112
2,common_voice_es_18306546.mp3_seg_0.wav,1.112
3,common_voice_es_18306547.mp3_seg_0.wav,2.462
4,common_voice_es_18306548.mp3_seg_0.wav,3.243
